# This notebook shows how to use Orchestrator APIs for user experiments

## Configure the Environment

In [1]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token_beta.json'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']='pruth_0031379841'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/FABRIC/pruth-bastion-1'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Setup the Experiment

#### Import FABRIC API

In [2]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import fablib

## Create Slice
In Release 1.0, user is expected to assign the IP addresses manually. Please use the example comands indicated below:

## Configure Slice Parameters



In [3]:
slice_name = 'MySliceL2Net'
site = 'RENC'
node1_name = 'Node1'
node2_name = 'Node2'
network_name='net1'
node1_nic_name = 'nic1'
node2_nic_name = 'nic2'
image = 'default_centos_8'
cores = 2
ram = 8
disk = 10

### Create the Slice

NIC component models options:
- NIC_Basic 
- NIC_ConnectX_5 
- NIC_ConnectX_6

In [4]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)

    # Node1
    node1 = slice.add_node(name=node1_name, site=site)
    node1.set_capacities(cores=cores, ram=ram, disk=disk)
    node1.set_image(image)
    iface1 = node1.add_component(model='NIC_ConnectX_5', name=node1_nic_name).get_interfaces()[0]
    #iface1.set_vlan(vlan=100)

    
    # Node2
    node2 = slice.add_node(name=node2_name, site=site)
    node2.set_capacities(cores=cores, ram=ram, disk=disk)
    node2.set_image(image)
    iface2 = node2.add_component(model='NIC_ConnectX_5', name=node2_nic_name).get_interfaces()[0]
    #iface2.set_vlan(vlan=100)

        
    # Network
    net1 = slice.add_l2network(name=network_name, interfaces=[iface1, iface2])

    #Submit Slice Request
    slice.submit(wait_progress=True)
except Exception as e:
    print(f"Slice Fail: {e}")
    traceback.print_exc()


Waiting for slice ............ Slice state: StableOK
Running post boot config ...Slice Fail: 'Node1'


Traceback (most recent call last):
  File "/var/folders/_l/8dq3pgzx6bjgt8qrr1m9jgc80000gp/T/ipykernel_16059/1039754221.py", line 25, in <module>
    slice.submit(wait_progress=True)
  File "/Users/pruth/Library/Python/3.9/lib/python/site-packages/fabrictestbed_extensions/fablib/slice.py", line 521, in submit
    self.post_boot_config()
  File "/Users/pruth/Library/Python/3.9/lib/python/site-packages/fabrictestbed_extensions/fablib/slice.py", line 380, in post_boot_config
    node.save_data()
  File "/Users/pruth/Library/Python/3.9/lib/python/site-packages/fabrictestbed_extensions/fablib/node.py", line 505, in save_data
    'os_interface':  i.get_physical_os_interface() }
  File "/Users/pruth/Library/Python/3.9/lib/python/site-packages/fabrictestbed_extensions/fablib/interface.py", line 109, in get_physical_os_interface
    return self.get_slice().get_interface_map()[network_name][node_name]
KeyError: 'Node1'


In [ ]:
try:

    print(f"interface_map: {slice.get_interface_map()}")
except Exception as e:
    print(f"Exception: \n{e}")

In [11]:
try:
    for reservtion_id,notice in slice.get_notices().items():
        print(f"reservtion_id: {reservtion_id}, notice: {notice}")
except Exception as e:
    print(f"Exception: \n{e}")
    

reservtion_id: 28d713f3-3707-4c44-8fe3-87bf916a3be8, notice: {'last_ticket_update': 'Insufficient resources : Component of type: ConnectX-5 not available in graph node: 3JB1R53', 'pending_state': 'None_', 'reservation_state': 'Closed'}
reservtion_id: 68f9611c-0feb-4c9d-84ce-cbc2d284e38f, notice: {'error_message': 'redeem predecessor reservation# 28d713f3-3707-4c44-8fe3-87bf916a3be8 is in a terminal state', 'pending_state': 'None_', 'reservation_state': 'Closed'}
reservtion_id: e29dd284-984d-40ed-81af-7b91c2b346a1, notice: {'last_ticket_update': 'TicketReviewPolicy: Closing reservation due to failure in slice', 'pending_state': 'None_', 'reservation_state': 'Closed'}


## Get the Slice

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"Slice: {slice.get_name()}")
except Exception as e:
    print(f"Get Slices Fail: {e}")

In [5]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   Cores             : {node.get_cores()}")
        print(f"   RAM               : {node.get_ram()}")
        print(f"   Disk              : {node.get_disk()}")
        print(f"   Image             : {node.get_image()}")
        print(f"   Image Type        : {node.get_image_type()}")
        print(f"   Host              : {node.get_host()}")
        print(f"   Site              : {node.get_site()}")
        print(f"   Management IP     : {node.get_management_ip()}")
        print(f"   Reservation ID    : {node.get_reservation_id()}")
        print(f"   Reservation State : {node.get_reservation_state()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
        print(f"   Components        :  ")
        for component in node.get_components():
            print(f"      Name             : {component.get_name()}")
            print(f"      Details          : {component.get_details()}")
            print(f"      Disk (G)         : {component.get_disk()}")
            print(f"      Units            : {component.get_unit()}")
            print(f"      PCI Address      : {component.get_pci_addr()}")
            print(f"      Model            : {component.get_model()}")
            print(f"      Type             : {component.get_type()}") 
        print(f"   Interfaces        :  ")
        for interface in node.get_interfaces():
            print(f"       Name                : {interface.get_name()}")
            print(f"           Bandwidth           : {interface.get_bandwidth()}")
            print(f"           VLAN                : {interface.get_vlan()}")          
            print(f"           OS Interface        : {interface.get_os_interface()}")       

except Exception as e:
    print(f"Fail: {e}")

Node:
   Name              : Node1
   Cores             : 2
   RAM               : 8
   Disk              : 10
   Image             : default_centos_8
   Image Type        : qcow2
   Host              : renc-w3.fabric-testbed.net
   Site              : RENC
   Management IP     : 152.54.15.48
   Reservation ID    : 5f413b14-01b0-4a15-b846-31fe9d524482
   Reservation State : Active
   SSH Command       : ssh -i /Users/pruth/.ssh/id_rsa -J pruth_0031379841@bastion-1.fabric-testbed.net centos@152.54.15.48
   Components        :  
      Name             : Node1-nic1
      Details          : Mellanox ConnectX-5 Dual Port 10/25GbE
      Disk (G)         : 0
      Units            : 1
      PCI Address      : ['0000:41:00.0', '0000:41:00.1']
      Model            : ConnectX-5
      Type             : SmartNIC
   Interfaces        :  
       Name                : Node1-nic1-p1
           Bandwidth           : 25
           VLAN                : None
SCP download fail. Slice: MySliceL2Net, Nod

## Get the Nodes

Retrieve the node information and save the management IP address.


### Configure Node1

Configure the interface on on  node1.  

```
ip addr add 192.168.1.10/24 dev eth1
```

In [ ]:
try:
    node1 = slice.get_node(name=node1_name)        
    node1_iface = node1.get_interface(network_name=network_name)  
    node1_iface.set_ip(ip="192.168.1.10", cidr="24")
    
    stdout, stderr = node1.execute(f'ip addr list {node1_iface.get_os_interface()}')
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

### Configure Node2

Configure the interface on on  node2.  

```
ip addr add 192.168.1.11/24 dev eth1
```

In [ ]:
try:
    node2 = slice.get_node(name=node2_name)        
    node2_iface = node2.get_interface(network_name=network_name)  
    node2_iface.set_ip(ip="192.168.1.11", cidr="24")
    
    stdout, stderr = node2.execute(f'ip addr list {node2_iface.get_os_interface()}')
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

## Delete Slice

In [10]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")